In [1]:
import numpy as np
import pandas as pd

### 1) une colonne motVedette (transformation/simplification de la prononciation d'origine) en utilisant le mapping du fichier Buchanan_PronChar_counts.csv

#### Import des données

In [2]:
# préciser le chemin du fichier à lire

cheminDuFichierDictionnaire = "./../data/dictionary/1757_Buchanan-J.csv"
cheminDuFichierMapping = "./../data/dictionary/Buchanan_PronChar_counts.csv" 

# demander à pandas de lire le fichier et garder ce tableau accessible dans une variable
dfDicoBuch = pd.read_csv(cheminDuFichierDictionnaire, delimiter=";", encoding="utf8")
dfDicoMapping = pd.read_csv(cheminDuFichierMapping, delimiter=";", encoding="utf8")

In [73]:
# afficher à l'écran un échantillon du tableau
dfDicoBuch.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId
9203,2,оaʹk_e_n,NaN,NaN,NaN,a.,5.0,149
11932,5,săbbătāʹrĭans,NaN,NaN,NaN,n.,3.0,187
6912,5,indĭʹſsŏlūble,L.,NaN,NaN,a.,33.0,117
6513,4,illāʹbŏrāte,L.,NaN,NaN,a.,68.0,111
831,2,ăſſuāʹge,NaN,NaN,NaN,v.,5.0,31


In [74]:
dfDicoMapping.sample(n=5)

,char,occ,graph
13,ā,2079,a
16,ŏ,2797,o
19,h,2383,h
38,k,717,k
1,ʹ,12725,NaN


#### Aucun caractère n'a plus d'une correspondance dans la table de mapping : 

In [3]:
(dfDicoMapping['char'].value_counts()>1).sum()

0

#### Transformation de la prononciation en mot

In [4]:
# suppression des lignes avec un caractère n'ayant pas de correspondance
dfMapping = dfDicoMapping[~dfDicoMapping['graph'].isna()].copy()

In [5]:
def pronToMot(strBuch, df_mapping):
    res = ""
    
    for charBuch in strBuch:
        dfRowFinded = df_mapping[df_mapping['char']==charBuch]

        if len(dfRowFinded==1):
            res+=str(dfRowFinded.iloc[0]['graph'])
        else:
            res+=''

    return res

In [6]:
## test
pronToMot('hauʹ_gh_ty', dfMapping)

'haughty'

In [10]:
dfDicoBuchMot = dfDicoBuch.copy()

dfDicoBuchMot['motVedette'] = dfDicoBuchMot.apply(lambda x: pronToMot(x['pron'], dfMapping),1)

In [11]:
dfDicoBuchMot

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19,abaft
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19,abaisance
3,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon
4,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned
...,...,...,...,...,...,...,...,...,...
15795,4,zōŏʹgrăphy,G.,NaN,NaN,n.,19.0,241,zoography
15796,4,zōŏʹphytès,G.,NaN,NaN,n.,20.0,241,zoophytes
15797,4,zōŏʹphŏric,G.,NaN,NaN,n.,21.0,241,zoophoric
15798,4,zōŏʹtŏmiſt,G.,NaN,NaN,n.,22.0,241,zootomist


In [12]:
cheminDicoBuchMot = "./../data/dictionary/DicoMotVedette.csv"
dfDicoBuchMot.to_csv(cheminDicoBuchMot, sep=";", encoding="utf8", index=False )

### 2) une colonne Sampa et une colonne Ipa à partir des mots vedettes obtenus (transformation/croisement de données) en utilisant ./../data/dataframe/BigiRich<s>Sample</s>.csv

In [13]:
pathDicoBigiRich = "./../data/dataframe/BigiRich.csv"
dfDicoBigiRich = pd.read_csv(pathDicoBigiRich, delimiter=";", encoding="utf8")

In [15]:
dfDicoBigiRich.sample(n=10)

,hwd,pronSampa,ipa
44387,galant,g A l @ n t,gɑlənt
70993,mackley,m { k l i:,mækliː
10138,berhow,b 3:r h @U,bɜ:rhoʊ
123037,vanausdall,v { n aU s d @ l,væn?sdəl
52546,hemorrhage,h E m 3:r i dZ,hɛmɜ:ri?
5082,ariana,A r\ i: { n @,ɑɹiːænə
93148,providing,p r\ @ v aI 4 i N,pɹəv?ɾiŋ
15382,built-in,b I l t i n,bɪltin
121218,unashamedly,V n @ S eI m @ d l i:,ʌnəʃ?mədliː
4080,anscom,@ n s k A m,ənskɑm


In [17]:
dfDicoBuch_f = dfDicoBuchMot.merge(dfDicoBigiRich, left_on = ['motVedette'], right_on= ['hwd']).drop(columns=['motVedette'])
dfDicoBuch_f

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,hwd,pronSampa,ipa
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus,{ b @ k @ s,æbəkəs
1,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon,@ b { n d @ n,əbændən
2,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned,@ b { n d @ n d,əbændənd
3,2,ăbăʹſh,F.,NaN,NaN,v.,9.0,19,abash,@ b { S,əbæʃ
4,2,ăbāʹte,F.,NaN,NaN,v.,10.0,19,abate,@ b eI t,əb?t
...,...,...,...,...,...,...,...,...,...,...,...
10328,2,zĕʹnith,NaN,NaN,NaN,n.,11.0,241,zenith,z i: n @ T,ziːnəθ
10329,1,zĕſt,NaN,NaN,NaN,n.,13.0,241,zest,z E s t,zɛst
10330,1,zĭnk,NaN,NaN,NaN,n.,14.0,241,zink,z I N k,zɪŋk
10331,3,zōʹdĭac,NaN,NaN,NaN,n.,16.0,241,zodiac,z @U 4 i: { k,zoʊɾiːæk


#### 2) a. isoler les mots de Buchanan qui n'existent pas dans Bigi. Tenter de comprendre les différentes causes et tenter d'expliquer comment résoudre les différents problèmes (bonus)

In [25]:
mots_manquants = [x for x in list(dfDicoBuchMot['motVedette']) if x not in list(dfDicoBuch_f['hwd'])]

In [26]:
# 15 800 - 10 333 mots manquants
len(mots_manquants)

5467

In [27]:
from random import sample

sample(mots_manquants, 10)

['cartouoch',
 'variegate',
 'futtle',
 'remonstrate',
 'calcination',
 'foresail',
 'lacerable',
 'ingratitude',
 'obediential',
 'ringbone']

Les mots de Buchanan n'existant pas dans Bigi sont des mots que nous n'avons pas pu correctement créé à partir de leur prononciation en première partie. Soit parce que la prononciation était mauvaise, soit parce que la table de mapping ne contenait pas les caractères présents dans la pronociation.

Pour résoudre ce problème, nous pourrions rechercher les mots de Bigi se rapprochant des mots vedettes de Buchanan générés à partir de leur prononciation. Par exemple, en utilisant la distance de Levenshtein qui est un bon indicateur de mots proches.

Ci-dessous, un exemple avec la librairie fuzzywuzzy, on voit que le mot 'accouontable' présent dans la liste des mots manquants est associé au mot 'accountable' avec un score de proximité de 96/100.

In [59]:
from fuzzywuzzy import process

mots_bigi = list(dfDicoBigiRich['hwd'])
highest = process.extractOne('accouontable', mots_bigi)
print(highest)

('accountable', 96)


Ci-dessous, un contre exemple montrant les limites de cette librairie avec le mot 'abetment'.

In [63]:
highest = process.extractOne('abetment', mots_bigi)
list_of_similar_words = process.extract('abetment', mots_bigi)
print(highest)
print(list_of_similar_words)

('a', 90)
[('a', 90), ('a.', 90), ('ab', 90), ('abe', 90), ('abet', 90)]


## 3) sauvegarder la nouvelle matrice obtenue dans ./../data/dataframe/BuchananRich.csv

In [65]:
cheminDuDfFinal = "./../data/dataframe/BuchananRich.csv"

dfDicoBuch_f.to_csv(cheminDuDfFinal, sep=";", encoding="utf8", index=False )